In [1]:
import pandas as pd
import numpy as np
from datetime import datetime 

In [2]:
data = pd.read_excel(r"C:\Users\User\Desktop\big blue\thalassa-mia-glende-exercises\Project2\Data\weatherOrg.xlsx")
pd.set_option('display.max_columns', None)
data.tail()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
361,2018-12-28,Cape Town,9.0,30.7,0.0,7.6,12.1,NNW,76.0,SSE,NW,7.0,50,38,15,1016.1,1010.8,1,3,20.4,30.0,No,0.0,No
362,2018-12-29,Cape Town,7.1,28.4,0.0,11.6,12.7,N,48.0,NNW,NNW,2.0,19,45,22,1020.0,1016.9,0,1,17.2,28.2,No,0.0,No
363,2018-12-30,Cape Town,12.5,19.9,0.0,8.4,5.3,ESE,43.0,ENE,ENE,11.0,9,63,47,1024.0,1022.8,3,2,14.5,18.3,No,0.0,No
364,2018-12-31,Cape Town,12.5,26.9,0.0,5.0,7.1,NW,46.0,SSW,WNW,6.0,28,69,39,1021.0,1016.2,6,7,15.8,25.9,No,0.0,No
365,NaT,NaN,12.3,30.2,0.0,6.0,12.6,NW,78.0,NW,WNW,31.0,35,43,13,1009.6,1009.2,1,1,23.8,28.6,No,0.0,No


In [3]:
df = data.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           365 non-null    datetime64[ns]
 1   Location       365 non-null    object        
 2   MinTemp        366 non-null    float64       
 3   MaxTemp        366 non-null    float64       
 4   Rainfall       366 non-null    float64       
 5   Evaporation    366 non-null    float64       
 6   Sunshine       363 non-null    float64       
 7   WindGustDir    363 non-null    object        
 8   WindGustSpeed  364 non-null    float64       
 9   WindDir9am     335 non-null    object        
 10  WindDir3pm     365 non-null    object        
 11  WindSpeed9am   359 non-null    float64       
 12  WindSpeed3pm   366 non-null    int64         
 13  Humidity9am    366 non-null    int64         
 14  Humidity3pm    366 non-null    int64         
 15  Pressure9am    366 non-

In [4]:
# Adding the last date, can't afford to drop it
df['Date'].iloc[365] = '2019-01-01'
df['Date'] = pd.to_datetime(df['Date'])

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [5]:
# Dropping unecessary columns 
df.drop(['Location','RainTomorrow'], axis = 1, inplace = True)

### Feature Engineering

After doing some research over the web and consalting kitesurf experts, we concluded that the weather conditions that we need to alert newbies for are the following:

1. Offshore winds
2. Wind speed above 30 km/h
3. Wind gust above 35 km/h
4. Rain 

In [7]:
# Kitesurfing 
df['Kitesurf'] = 'Yes'

# ENE is the offshore wind based on the location of our beach 
for i in range(0, 327):
    if (df['WindGustDir'].iloc[i] == 'ENE') | (df['WindDir9am'].iloc[i] == 'ENE') | (df['WindDir3pm'].iloc[i] == 'ENE'):
        df['Kitesurf'].iloc[i] = 'No'       

for i in range(0, 327):
    if (df['WindSpeed9am'].iloc[i] > 30) | (df['WindSpeed3pm'].iloc[i] > 30):
        df['Kitesurf'].iloc[i] = 'No'
        
for i in range(0, 327):
    if (df['WindGustSpeed'].iloc[i] > 35):
        df['Kitesurf'].iloc[i] = 'No'
        
for i in range(0, 327):
    if (df['RainToday'].iloc[i] == 1):
        df['Kitesurf'].iloc[i] = 'No'

In [8]:
df.head()

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,Kitesurf
0,2018-01-01,8.0,24.3,0.0,3.4,6.3,NW,30.0,SW,NW,6.0,20,68,29,1019.7,1015.0,7,7,14.4,23.6,No,3.6,Yes
1,2018-01-02,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,W,4.0,17,80,36,1012.4,1008.4,5,3,17.5,25.7,Yes,3.6,No
2,2018-01-03,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,NNE,6.0,6,82,69,1009.5,1007.2,8,7,15.4,20.2,Yes,39.8,No
3,2018-01-04,13.3,15.5,39.8,7.2,9.1,NW,54.0,WNW,W,30.0,24,62,56,1005.5,1007.0,2,7,13.5,14.1,Yes,2.8,No
4,2018-01-05,7.6,16.1,2.8,5.6,10.6,SSE,50.0,SSE,ESE,20.0,28,68,49,1018.3,1018.5,7,7,11.1,15.4,Yes,0.0,No


My df is clean and I can't afford to drop the NaNs. Saving the data

In [9]:
df.to_csv(r"C:\Users\User\Desktop\big blue\thalassa-mia-glende-exercises\Project2\Data\clean_data", index = False)